# Deep Learning Project Report

Authors: 
-   Francesco Vaccari   francesco.vaccari@studenti.unitn.it [239927]
-   Gabriele Stulzer    gabriele.stulzer@studenti.unitn.it  []
-   Nicola Maestri      ??  []

### Index
-   [Abstract](#abstract)
-   [Introduction](#introduction)
-   [Related Work](#related-work)


Sarebbe anche da spiegare dove abbiamo messo il codice, se lo abbiamo messo nell'appendice o tutto nel testo

### Abstract

Visual Grounding is a challenging task which aims to locate an object in an image based on a natural language query. Despite impressive advances in computer vision and natural language processing, establishing meaningful connections between distinct elements of images and text to get a good comprehension of context is still a big research area. In our work, we explored some new strategies to solve the problem by laveraging the image-text alignment of Clip as a foundation model for new frameworks specialized in Visual Grounding.

### Introduction

First, we propose a baseline that combines the object detector Yolo [1] and CLIP [2] model without any other component. For each candidate object proposed by Yolo is computed a cosine similarity with the text query and the most similar one is chosen as output prediction. We then tried to break free from the object detector developing a framework based on heatmaps [3] to retrieve the target object and on SAM [4] to draw a proper bounding box. We finally developed a new framework obtained by customizing the CLIP model to compute a low-level probability map from which we find a bounding box through some up-sampling convolutions [5]. Fine-tuning allowed us to exploit Clip strengths without the need for heavy training, therefore beyond the results, it seems to be the most promising direction.

### Related Work

- Visual Grounding
- Reffering Image Segmentation
- Adapters and Fine-tuning

## Dataset

## Baseline

## SAM

## RISCLIP

In [5] it is proposed an interesting approach that adapts CLIP for the task of Referring Image Segmentation (RIS). The authors add in the model of CLIP Fusion and Backbone Adapters to improve RIS capablities, doing so allows to only train the block introduced while keeping the original CLIP framework frozen. This allows to exploit the rich and general image-text alignment that CLIP offers and is not as computationally expesinve as performing a full fine-tuning of the model. The Fusion adapters have the objective of allowing multi-modal communication between the visual and textual layers of transformers, while the Backbone Adapters inject new knowledgefor solving the task at hand.

In our project we replicated the structure of the model described in the paper [5] for the ViT-B/16 version of CLIP and adapted it to the task of Visual Grounding.

Qua andrebbe spiegato cosa abbiamo cambiato nel refiner per fargli fare visual grounding, sempre se abbiamo cambiato robe. Perché non sono sicuro che abbiamo cambiato qualcosa, forse semplicemente creiamo il bounding box attorno alla segmentation.

File per il modello: adapters, refiner, model, clip, simple_tokenizer. Di questi parlare solo dei primi tre, il resto non è importante.

Del dataset in teoria ne abbiamo già parlato sopra quindi non serve.

File per training: focaldiceloss, training_locator, training_refiner

File per inference/evaluation: tester

Infine risultati


## Conclusion

## References

1. Redmon, Joseph, Santosh Divvala, Ross Girshick, and Ali Farhadi. "You only look once: Unified, real-time object detection." In Proceedings of the IEEE conference on computer vision and pattern recognition, pp. 779-788. 2016.

2. Radford, Alec, Jong Wook Kim, Chris Hallacy, Aditya Ramesh, Gabriel Goh, Sandhini Agarwal, Girish Sastry et al. "Learning transferable visual models from natural language supervision." In International conference on machine learning, pp. 8748-8763. PMLR, 2021.

3. Li, Junnan, Dongxu Li, Caiming Xiong, and Steven Hoi. "Blip: Bootstrapping language-image pre-training for unified vision-language understanding and generation." In International Conference on Machine Learning, pp. 12888-12900. PMLR, 2022.

4. Kirillov, Alexander, Eric Mintun, Nikhila Ravi, Hanzi Mao, Chloe Rolland, Laura Gustafson, Tete Xiao et al. "Segment anything." arXiv preprint arXiv:2304.02643 (2023).

5. Kim, Seoyeon, Minguk Kang, and Jaesik Park. "RISCLIP: Referring Image Segmentation Framework using CLIP." arXiv preprint arXiv:2306.08498 (2023).

6. Vaswani, Ashish, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin. "Attention is all you need." Advances in neural information processing systems 30 (2017).


Per aggiungere una citazione andare su https://scholar.google.com , cercare il nome del paper, una volta trovato premere sul pulsantino "Cite" e copiare la citazione nello stile Chicago. Si prega di non cambiare i numeri già messi

## Appendix

Qua pensavo di metterci tutto quello che non mettiamo direttamente nel report, quindi codice che abbiamo scritto/modificato che però non è il punto centrale. Ci possiamo mettere anche risultati secondari.